# 第4回講義 宿題

## 課題

今Lessonで学んだことを元に，MNISTのファッション版 (Fashion MNIST，クラス数10) を多層パーセプトロンによって分類してみましょう．

Fashion MNISTの詳細については以下のリンクを参考にしてください．

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値

Accuracy 85%

### ルール


- 訓練データは`x_train`， `t_train`，テストデータは`x_test`で与えられます．
- 予測ラベルは one_hot表現ではなく0~9のクラスラベル で表してください．
- **下のセルで指定されている`x_train`，`t_train`以外の学習データは使わないでください．**
- Pytorchを利用して構いません．
- ただし，**`torch.nn.Conv2d`のような高レベルのAPIは使用しないで下さい**．具体的には，`nn.Parameter`, `nn.Module`, `nn.Sequential`以外の`nn`系のAPIです．使用した場合エラーになります．
- torchvision等で既に実装されているモデルも使用しないで下さい．

### 提出方法
- 2つのファイルを提出していただきます．
    1. テストデータ (`x_test`) に対する予測ラベルをcsv形式で保存し，**Omnicampusの宿題タブから「第4回 ニューラルネットワークの最適化・正則化」を選択して**提出してください．
    2. それに対応するpythonのコードを　ファイル＞ダウンロード＞.pyをダウンロード　から保存し，**Omnicampusの宿題タブから「第4回 ニューラルネットワークの最適化・正則化 (code)」を選択して**提出してください．pythonファイル自体の提出ではなく，「提出内容」の部分にコード全体をコピー&ペーストしてください．
      
- なお，採点は1で行い，2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）．コードの内容を変更した場合は，**1と2の両方を提出し直してください**．

### 評価方法
- 予測ラベルの`t_test`に対する精度 (Accuracy) で評価します．
- 即時採点しLeader Boardを更新します（採点スケジュールは別アナウンス）．
- 締切時の点数を最終的な評価とします．

### ドライブのマウント

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# 作業ディレクトリを指定
# work_dir = 'drive/MyDrive/Colab Notebooks/DLBasics2025_colab'

### データの読み込み（この部分は修正しないでください）

`__len__`は，Pythonの組み込み関数len()を呼んだときに，内部で呼ばれる特殊メソッドです．

`__getitem__`は，インデックスやキーで要素を取得するときに，内部で呼ばれる特殊メソッドです．

どちらも， Datasetクラスを自作する際によく登場します．

```python
class MyList:
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

mylist = MyList([10, 20, 30])
print(len(mylist))  # __len__が呼び出される
# 3
print(mylist[1])  # __getitem__が呼び出される
# 20
```

In [ ]:
# API制限のコードと，torchに統合されているdynamoライブラリの挙動がコンフリクトを起こすため，dynamoを無効化
import torch._dynamo
torch._dynamo.disable()

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import inspect

# 使用できるAPIを制限
nn_except = ["Module", "Parameter", "Sequential"]
for m in inspect.getmembers(nn):
    if not m[0] in nn_except and m[0][0:2] != "__":
        delattr(nn, m[0])

seed = 1234
torch.manual_seed(seed)
np.random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 学習データ
x_train = np.load('../data/x_train.npy')
t_train = np.load('../data/y_train.npy')

# テストデータ
x_test = np.load('../data/x_test.npy')

class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train):
        self.x_train = x_train.reshape(-1, 784).astype('float32') / 255
        self.t_train = t_train

    def __len__(self):
        return self.x_train.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_train[idx], dtype=torch.float), torch.tensor(self.t_train[idx], dtype=torch.long)

class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_test):
        self.x_test = x_test.reshape(-1, 784).astype('float32') / 255

    def __len__(self):
        return self.x_test.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_test[idx], dtype=torch.float)

trainval_data = train_dataset(x_train, t_train)
test_data = test_dataset(x_test)

### 多層パーセプトロンの実装

In [ ]:
batch_size = 32

val_size = 10000
train_size = len(trainval_data) - val_size

train_data, val_data = torch.utils.data.random_split(trainval_data, [train_size, val_size])

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=False
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
def relu(x, alpha=0.01):
    x = torch.where(x < 0, alpha*x, x)
    return x


def softmax(x):
    x_stable = x - torch.cat([x.max(axis=1, keepdim=True).values] * x.size()[1], dim=1)
    x_exp = torch.exp(x_stable)
    return x_exp/torch.cat([x_exp.sum(dim=1, keepdim=True)] * x.size()[1], dim=1)


def dropout(x, p=0.5, training=True):
    if not training or p == 0.0:
        return x
    mask = (torch.rand_like(x) > p).float()/(1.0 - p)
    return x * mask


class Dense(nn.Module):  # nn.Moduleを継承する
    def __init__(self, in_dim, out_dim, p, function=lambda x: x,):
        super().__init__()
        self.W = nn.Parameter(torch.tensor(np.random.uniform(
            low=-np.sqrt(6 / (in_dim)),
            high=np.sqrt(6 / (in_dim)),
            size=(in_dim, out_dim)
        ).astype('float32')))
        self.b = nn.Parameter(torch.zeros(out_dim, dtype=torch.float32))
        self.function = function
        self.p = p

    def forward(self, x):
        return self.function(dropout(torch.matmul(x, self.W) + self.b), self.p, self.training)


class MLP(nn.Module):  # nn.Moduleを継承する
    def __init__(self, in_dim, hid_dim1, hid_dim2, out_dim, p1, p2):
        super().__init__()
        self.linear1 = Dense(in_dim, hid_dim1, p1, function=relu)
        self.linear2 = Dense(hid_dim1, hid_dim2, p2, function=relu)
        self.linear3 = Dense(hid_dim2, out_dim, 0, function=softmax)

    def forward(self, x):
        x = self.linear1.forward(x)
        x = self.linear2.forward(x)
        return x

in_dim = 784
hid_dim1 = 512
hid_dim2 = 256
out_dim = 10
lr = 0.001
n_epochs = 10
p1 = 0.2
p2 = 0.5


mlp = MLP(in_dim, hid_dim1, hid_dim2, out_dim, p1, p2).to(device)

optimizer = optim.Adam(mlp.parameters(), lr=lr)

dataloader_train = torch.utils.data.DataLoader(x_train, batch_size=batch_size, shuffle=True)
dataloader_valid = torch.utils.data.DataLoader(x_train, batch_size=batch_size, shuffle=False)

In [ ]:
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []
    train_num = 0
    train_true_num = 0
    valid_num = 0
    valid_true_num = 0

    mlp.train()  # 訓練時には勾配を計算するtrainモードにする
    for x, t in dataloader_train:
        t_hot = torch.eye(10)[t]

        # move data to gpu
        x = x.to(device)
        t_hot = t_hot.to(device)

        y = mlp.forward(x)
        loss = -(t_hot * torch.log(y+ 1e-8)).sum(dim=1).mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pred = y.argmax(1)

        losses_train.append(loss.tolist())

        acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
        train_num += acc.size()[0]
        train_true_num += acc.sum().item()

    mlp.eval()  # 評価時には勾配を計算しないevalモードにする
    for x, t in dataloader_valid:
        t_hot = torch.eye(10)[t]

        x = x.to(device)
        t_hot = t_hot.to(device)

        y = mlp.forward(x)

        loss = -(t_hot * torch.log(y)).sum(dim=1).mean()

        pred = y.argmax(1)

        losses_valid.append(loss.tolist())

        acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
        valid_num += acc.size()[0]
        valid_true_num += acc.sum().item()

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        train_true_num/train_num,
        np.mean(losses_valid),
        valid_true_num/valid_num
    ))

In [ ]:
mlp.eval()

t_pred = []
for x in dataloader_test:

    x = x.to(device)

    # 順伝播
    y = mlp.forward(x)

    # モデルの出力を予測値のスカラーに変換
    pred = y.argmax(1).tolist()

    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv(work_dir + '/Lecture04/submission_pred_04.csv', header=True, index_label='id')